In [ ]:
from PIL import Image
from IPython.display import display

import PyNvCodec as nvc
import numpy as np
import os

In [ ]:
def make_img(buffer,w,h,format,merge=False):
    """
    utility function to build image from buffers. requires numpy, pillow. 
    
    Parameters
    ----------
    buffer:  Numpy array
    w, h :   Width and height of image, regardeless of buffer format
    format : Format of buffer. may be RGB, RGB_Planar, NV12, YUV420p
    merge :  Whether we should reconstitute rgb image or display the whole buffer (as grayscale image)
    """
    
    match format:
        case 'RGB':
            return Image.fromarray(buffer.reshape(h,w,3))
        case 'RGB_Planar':
            if not merge:
                return Image.fromarray(buffer.reshape(h*3,w))
            else:
                return Image.fromarray(buffer.reshape(3,h,w).transpose(2,1,0)).transpose(Image.TRANSPOSE)
        case 'NV12':
            if not merge:
                return Image.fromarray(buffer.reshape(h*3//2,w))
            else:
                y=Image.fromarray(buffer[:w*h].reshape(h,w))
                u=Image.fromarray(buffer[w*h:][::2].reshape(h//2,w//2).repeat(2, axis=0).repeat(2, axis=1))
                v=Image.fromarray(buffer[w*h+1:][::2].reshape(h//2,w//2).repeat(2, axis=0).repeat(2, axis=1))
                return Image.merge('YCbCr', (y,u,v))
        case 'YUV420p':
            if not merge:
                return Image.fromarray(buffer.reshape(h*3//2,w))
            else:
                y=Image.fromarray(buffer[:w*h].reshape(h,w))
                u=Image.fromarray(buffer[w*h:w*h*5//4].reshape(h//2,w//2).repeat(2, axis=0).repeat(2, axis=1))
                v=Image.fromarray(buffer[w*h*5//4:].reshape(h//2,w//2).repeat(2, axis=0).repeat(2, axis=1))
                return Image.merge('YCbCr', (y,u,v))

In [ ]:
class TwoPassConverter:
    def __init__(self,fmt,w,h):
        self.fmt,self.w,self.h=fmt,w,h
        self.to_yuv=nvc.PySurfaceConverter(fmt,nvc.PixelFormat.YUV420,gpu_id=0)
        self.to_rgb=nvc.PySurfaceConverter(nvc.PixelFormat.YUV420,nvc.PixelFormat.RGB,gpu_id=0)
        self.surf_yuv = nvc.Surface.Make(format=self.fmt, width=self.w,height=self.h,gpu_id=0)

    def Run(self,src,dst,cc_ctx):
        self.surf_yuv = nvc.Surface.Make(format=self.fmt, width=self.w,height=self.h,gpu_id=0)
        success,details=self.to_yuv.Run(src,self.surf_yuv,cc_ctx)
        if not success:
            return success, details
        success,details=self.to_rgb.Run(self.surf_yuv,dst,cc_ctx)
        return success, details

class OnePassConverter:
    def __init__(self,fmt,w,h):
        self.fmt, self.w, self.h=fmt, w, h
        self.to_rgb=nvc.PySurfaceConverter(fmt,nvc.PixelFormat.RGB,gpu_id=0)
        
    def Run(self,src,dst,cc_ctx):
        success,details=self.to_rgb.Run(src,dst,cc_ctx)
        return success, details

def check(success, info):
    if not success:
        print(sys._getframe(1).f_lineno)
        print(info)

In [ ]:
class StopExecution(Exception):
    def _render_traceback_(self):
        return []

In [ ]:
url="/home/yves/Devel/python/vali_luma_chroma_shift/hires.mp4"
#url=f"{os.environ['VALI']}/tests/data/test.mp4"
sq=256 if 'lores' in url else 512

In [ ]:
# GPU-accelerated decoder
pyDec = nvc.PyDecoder( url, {}, gpu_id=0)
assert (pyDec.Format()==nvc.PixelFormat.NV12)
cvt=OnePassConverter(pyDec.Format(),pyDec.Width(),pyDec.Height())
cc_ctx = nvc.ColorspaceConversionContext(nvc.ColorSpace.BT_709,nvc.ColorRange.MPEG)
pyDwn = nvc.PySurfaceDownloader(gpu_id=0)

In [ ]:
# Raw decoded Surface
surf_src = nvc.Surface.Make(format=pyDec.Format(),       width=pyDec.Width(), height=pyDec.Height(), gpu_id=0)
surf_dst = nvc.Surface.Make(format=nvc.PixelFormat.RGB,  width=pyDec.Width(), height=pyDec.Height(), gpu_id=0)

# Numpy array which contains decoded RGB Surface
rgb_frame = np.ndarray(dtype=np.uint8,shape=surf_dst.HostSize())
nv_frame  = np.ndarray(dtype=np.uint8,shape=pyDec.Width()*pyDec.Height()*3//2)

for _ in range (10):
    assert (pyDec.Format()==nvc.PixelFormat.NV12)

    # Decode single Surface
    success, info = pyDec.DecodeSingleSurface(surf_src)

    success, info = pyDwn.Run(surf_src, nv_frame)        ## get raw decoded frame, later decoded by pillow!
    success, info = cvt.Run(surf_src,surf_dst,cc_ctx)    ## convert to rgb! 
    success, info = pyDwn.Run(surf_dst, rgb_frame)       ## get rgb converted frame

    res_frame = np.reshape(rgb_frame,(pyDec.Height(), pyDec.Width(), 3))
    display(Image.fromarray(res_frame))#.crop((pyDec.Width()//2,pyDec.Height()//2,sq+pyDec.Width()//2,sq+pyDec.Height()//2)))#.resize((320,160)))

    res_img=make_img(nv_frame,pyDec.Width(),pyDec.Height(),'NV12',merge=True)
    display(res_img)#.crop((pyDec.Width()//2,pyDec.Height()//2,sq+pyDec.Width()//2,sq+pyDec.Height()//2)))#.resize((320,160)))